In [2]:
L = 8
t = 1
function Hamiltonian(L,t)
    H = zeros(2*L, 2*L)
    H[1,L] = -t
    H[L,1] = -t
    H[2L,L+1] = -t
    H[L+1,2L] = -t
    for i=1:2*L
      for j=1:2*L
        if i == j + 1 || i == j - 1
          H[i,j] = -t
        end 
      end
    end
    H[L,L+1]=0
    H[L+1,L]=0
    return H
end

H = Hamiltonian(L,t)

16×16 Matrix{Float64}:
  0.0  -1.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   0.0  -1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   0.0  -1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   0.0  -1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0  -1.0
  0.0   0.0   0.0   0.0   0.0   0.0     -1.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  …   0.0  -1.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0     -1.0   0.0  -1.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0  -1.0   0.0  -1.0   0.0   0.0
  

In [3]:
using LinearAlgebra
U = eigvecs(H)  #unitary matrix
Udag = inv(U)   #U^-1
E = eigvals(H)  #eigenvalues of H

-2.0000000000000004

In [8]:
function U1(A, Ne) #take the first Ne columns
    U_1 = A[:,1:4]
    return U_1
end

U_1 = U1(U,4)

16×4 Matrix{Float64}:
 -0.353553   0.0        0.353553      0.0
 -0.353553   0.0        0.5           0.0
 -0.353553   0.0        0.353553      0.0
 -0.353553   0.0       -5.55112e-17   0.0
 -0.353553   0.0       -0.353553      0.0
 -0.353553   0.0       -0.5           0.0
 -0.353553   0.0       -0.353553      0.0
 -0.353553   0.0        0.0           0.0
  0.0       -0.353553   0.0           0.353553
  0.0       -0.353553   0.0           0.5
  0.0       -0.353553   0.0           0.353553
  0.0       -0.353553   0.0          -5.55112e-17
  0.0       -0.353553   0.0          -0.353553
  0.0       -0.353553   0.0          -0.5
  0.0       -0.353553   0.0          -0.353553
  0.0       -0.353553   0.0           0.0

In [32]:
function occupied_states(v)
    occ_states = []
    for i = 1 : 2*L
        if v[i] == 1
            append!(occ_states, i )
        end
    end
    return occ_states
end
    

function U_tilde(A,v)                  #takes just the rows related to occupied states, Ne*Ne matrix
    U_tild = []
    U_tild = A[occupied_states(v),:]
    return U_tild
end

U_tilde (generic function with 1 method)

In [71]:
state = [1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0]      #initial state

U_2 = U_tilde(U_1,state)                       #initial U tilde

state_1 = [1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0]    #|x'>

U_3 = U_tilde(U_1,state_1)                     #new U tilde

ratio = det(U_3) / det(U_2)                    #ratio of determinants of U tilde matrices

1.2071067811865481

In [72]:
#Jastrow factor
Jastrow_ratio = 1

1

In [73]:
function MC(q0=0,delta=0.1,nsteps=1000,seed=None):
    if seed is not None:
        np.random.seed(seed)
    q=q0
    traj=[]
    for istep in range(nsteps):
        qtry=q+(2*np.random.rand()-1)*delta
        energy_try=0.5*kappa*qtry**2     #1/2 k q^2
        deltae=energy_try-energy
        if deltae<=0.0:          #take acceptance 1 if the energy decrease
            acceptance=1.0
        else:
            acceptance =  ratio**2 * Jastrow_ratio**2   #acceptance 
        if acceptance >=1.0 or acceptance>np.random.rand(): #metropolis rule
            q=qtry
            energy=energy_try
        traj.append(q)
    return np.array(traj)

LoadError: syntax: newline not allowed after ":" used for quoting

In [76]:
function move(v)
    x = occupied_states(v)
    i = rand( 1:length(x))
    print(x[i])
    v[x[i]] = 0
    if x[i] < length(x)
        v[x[i]+2*rand(0:1)-1] += 1
    end
    return v
end
state111 = move(state)

1

LoadError: BoundsError: attempt to access 16-element Vector{Int64} at index [0]

In [61]:


function MC
    
    
    
    acceptance =  ratio**2 * Jastrow_ratio**2   #acceptance 
    if acceptance >=1.0 or acceptance>rand(): #metropolis rule
        

LoadError: syntax: expected "end" in definition of function "MC"